In [3]:
import warnings

warnings.filterwarnings("ignore")

import pyspark.sql
from pyspark.sql.functions import *

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from wrangle import wrangle_311

spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [4]:
df = wrangle_311(spark)
print("\ndf shape: (%d, %d)\n" % (df.count(), len(df.columns)))
df.show(1, vertical=True)

[wrangle.py] reading case.csv
[wrangle.py] handling data types
[wrangle.py] parsing dates
[wrangle.py] adding features
[wrangle.py] joining departments

df shape: (841704, 20)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 219                  
 days_to_closed       | 0                

Answer the questions below by using a combination of the techniques discussed in the lesson that you think is appropriate.
### How many different cases are there, by department?

In [7]:
df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = false)
 |-- num_weeks_late: double (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- case_age: integer (nullable = true)
 |-- days_to_closed: integer (nullable = true)
 |-- case_lifetime: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- dept_subject_to_SLA: boolean (nullable = true)



In [20]:
df.groupBy('department').agg(count('case_id').alias('count')).sort(desc('count')).show(truncate=False)

+------------------------+------+
|department              |count |
+------------------------+------+
|DSD/Code Enforcement    |321374|
|Solid Waste             |279270|
|Animal Care Services    |116915|
|Trans & Cap Improvements|96193 |
|Parks & Recreation      |19907 |
|Metro Health            |5163  |
|Customer Service        |2849  |
|City Council            |33    |
+------------------------+------+



### Does the percentage of cases that are late vary by department?

In [40]:
(df.where(df.case_late == True).count()) / df.select(df.case_late).count()

0.11227581192438196

In [37]:
df.select((df.where(df.case_late == True).count()) / (df.select(df.case_late)).count()).show()

TypeError: Invalid argument, not a string or column: 0.11227581192438196 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

### On average, how late are the late cases by department?

### What is the service type that is the most late? Just for Parks & Rec?

### For the DSD/Code Enforcement department, what are the most common service request types? Look at other departments too.

### Does whether or not its a weekend matter for when a case is opened/closed?

### On average, how many cases are opened a day for the Customer Service department?

### Does the number of service requests for the solid waste department vary by day of the week?